In [1]:
import os                          # Import operating system interface
import win32com.client as win32    # Import COM
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import optuna as opt
import dill
import time
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice
from tqdm.notebook import tqdm
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.axisbelow'] = True
plt.rcParams['figure.figsize'] = [7, 5]  # Largo, ancho  
plt.rcParams["text.usetex"] = False
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import pymoo as pym
import platform
print('Python version', platform.python_version())
print('Numpy version', np.__version__)
print('Optuna version',opt.__version__)
print('Pymoo version', pym.__version__)
print('Seaborn',sns.__version__)
from pathlib import Path
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Binary
from pymoo.algorithms.moo.nsga2 import NSGA2 #, RankAndCrowdingSurvival
from pymoo.core.mixed import MixedVariableMating, MixedVariableGA, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.termination import get_termination
from pymoo.termination.max_gen import MaximumGenerationTermination

Python version 3.11.1
Numpy version 1.26.4
Optuna version 3.6.1
Pymoo version 0.6.1.1
Seaborn 0.13.0


# Optimization using NSGA II

### Optimization of pressure of the entire system.

The final cost of the process is still higher at standar conditions as the other desing integrated (base case, RD , DW). Pressure may be a key feature that must be optmized as well and in this part is going to be integrated in the following code.

The Class  IntensifiedSequence  will be redefined. The __ini__ function is modified by adding the new parameter. 

In [2]:

class IntensifiedSequence_Pressure(ElementwiseProblem):
    
    def __init__(self, **kwargs):
        variables = dict()
        # ------------------Bounds----------------------#
        # First column
        Max_NT_B1, Max_NT_B2 = 75 , 75
        Max_Rr_B2 = 3.0
        variables[f"x1"] = Integer(bounds=(1,Max_NT_B1))      # FEED STAGE S1
        variables[f'x2'] = Integer(bounds=(1,Max_NT_B1))      # FEED STAGE S2
        variables[f'x3'] = Integer(bounds=(10,Max_NT_B1))      # ETAPAS B1
        variables[f'x4'] = Integer(bounds=(30,Max_NT_B2))     # ETAPAS B2
        variables[f'x5'] = Real(bounds=(0.05,Max_Rr_B2))      # REFLUJO B2
        variables[f'x6'] = Integer(bounds=(2,Max_NT_B2))      # Posicion en la que inicia la pared divisoria
        variables[f'x7'] = Integer(bounds=(2,Max_NT_B2))      # Posición de donde sale la corriente lateral
        variables[f'x8'] = Real(bounds=(1,100))               # PROD_FLOW_L2 
        variables[f'x9'] = Real(bounds=(1,300))              # PROD_FLOW_V3
        variables[f'x10'] = Real(bounds= (4.5,8.0) )          # PRESSURE OF THE PROCESS
        super().__init__(vars=variables, n_obj=4,n_ieq_constr=1, **kwargs)

    #-----------------------------------------------------#
    #  FUNCTIONS TO CALCULATE CAPITAL AND UTILITIES COST  #
    #-----------------------------------------------------#

    def Column(self, nt1,  ds , P, payback = 3, M_S = 1716, fm = 3.67, ft = 0):
        # Inputs
        # M_S : M&S is Marshall and Swift Equipment Cost Index year 2019
        # fm value for stainless steel
        # d : divided column diameter  in meters
        # nt : number of trays
        # ft : tray sieve
        # Variables
        # L : column length in meters  
        H = 1.2*0.61*(nt1-2)
        fp = 1 + 0.00074*(P - 3.48) + 0.00023*(P-3.48)**2
        fc = fm*fp
        CostColumn =  (M_S/280)*(957.9*(ds**1.066)*(H**0.802))*(2.18 + fc) 
        PlatosColumn =  (M_S/280)*(round(nt1))*97.2*(ds**1.55)*(ft + fm)
        TotalCost = (CostColumn + PlatosColumn)/payback   # [USD/yr]
        return TotalCost
    
    def CondenserColumn(self,COND_DUTY,TOP_TEMP, payback = 3, labour = 8000, M_S = 1716, \
        ktc =  0.852,fd = 0.8,fm =2.81, fp = 0 ):
        # Inputs
        # M_S : M&S is Marshall and Swift Equipment Cost Index year 2019
        # ktc : Heater-transfer coefficient  in kW/m2K
        # Fm  : carbon steel shell, stainless steel tube
        # Fd  : fixed tube sheet
        # Fp  : pressures lower than 10.3 bar  
        # --------------------------#
        # Accessing values of ASPEN #
        # --------------------------#
        Condenserduty = Application.Tree.FindNode(COND_DUTY).Value*0.0041868  # cal/sec to kW
        Tcond  = Application.Tree.FindNode(TOP_TEMP).Value + 273.15 #Kelvyn
        # --------------------------#
        CoolingService = [278, 310]  # [K] Chilled water, cooling water # 5 y 36.85°C
        Utilitiprice = [4.43, 0.354] # [$/GJ]
        if CoolingService[1] < Tcond:  
            # Use cooling water
            Tcools = CoolingService[1]
            Uprice = Utilitiprice[1]
        elif CoolingService[0] < Tcond:
            # Use chilled water
            Tcools = CoolingService[0]
            Uprice = Utilitiprice[0]
        # ------------ #
        # Capital Cost #
        # ------------ #
        DeltaT_Condenser = Tcond - Tcools 
        AreaCondenser = abs(Condenserduty)/(ktc*(DeltaT_Condenser)) # [m2]
        fc = (fd+fp)*fm
        K = M_S*1.695*(2.29 + fc)
        CostCondenser = K*(AreaCondenser**0.65)
        # --------------- #
        # Utilititie Cost #
        # --------------- #     
        UtiCondenser = Uprice*abs(Condenserduty)*(1e-6)*labour*3600  # [$/year]        
        TotalCost = CostCondenser/payback + UtiCondenser  # [$/year]
        return TotalCost

    def ReboilerColumn(self,REB_DUTY,BOTTOM_TEMP, payback = 3, labour = 8000, M_S = 1716, \
        ktc =  0.568,fd =1.35,fm=2.81, fp = 0):
        # Inputs
        # BOTTOM_TEMP, REB_DUTY : direction of parameters in ASPEN 
        # payback, labour : payback period and labour hours
        # M_S : M&S is Marshall and Swift Equipment Cost Index year 2019
        # ktc : Heater-transfer coefficient  in kW/m2K
        # Fd  : fixed tube sheet
        # Fm  : carbon steel shell, stainless steel tube
        # Fp  : 0 (pressures < 10.3 bar default value)
        # --------------------------#
        # Accessing values of ASPEN #
        # --------------------------#
        Temp  = Application.Tree.FindNode(BOTTOM_TEMP).Value + 273.15 # Celsius to Kelvyn
        ReboilerDuty = Application.Tree.FindNode(REB_DUTY).Value*0.0041868    #cal/sec to kW

        ReboilerService = [433,457,527]  # Tempearture [K]
        Utilitiprice = [7.78,8.22,9.8]     # [$/GJ]
        LatenHeatService = [2085.03,1998.55,1697.79] #KJ/Kg
        if Temp < ReboilerService[0]:       #Use low pressure steam 
            L = LatenHeatService[0]
            Uprice = Utilitiprice[0]
            Treb = ReboilerService[0]
        elif Temp < ReboilerService[1]:     #Use medium pressure steam
            L = LatenHeatService[1]
            Uprice = Utilitiprice[1]
            Treb = ReboilerService[1]
        elif Temp < ReboilerService[2]:     #Use high pressure steam
            L = LatenHeatService[2]
            Uprice = Utilitiprice[2]
            Treb = ReboilerService[2]

        # ------------ #
        # Capital Cost #
        # ------------ #
        fc = (fd + fp)*fm
        DeltaT_Reboiler = Treb - Temp   # Steam temperature-base temperature
        AreaReboiler = ReboilerDuty/(DeltaT_Reboiler*ktc)
        K = M_S*1.695*(2.29 + fc)
        CostReboiler = K*(AreaReboiler**0.65)  #[$]
        # --------------- #
        # Utilititie Cost #
        # --------------- #
        UtiReboiler = Uprice*ReboilerDuty*(1e-6)*labour*3600  # [$/year]
        TotalCost = CostReboiler/payback + UtiReboiler  # [$/year]
        return TotalCost

    def Antoine_function(self,Antoine1,Antoine2,Antoine3,Antoine4,Antoine5,Antoine6,Antoine7,T):
        ListPsat = [] #np.zeros(componentes)
        for i in range(len(Antoine1)):
            ListPsat.append(np.exp(Antoine1[i] + Antoine2[i]/(T+Antoine3[i]) + Antoine4[i]*T + Antoine5[i]*np.log(T) +  Antoine6[i]*T**Antoine7[i]))
        return np.array(ListPsat)

    def Antoine_values(self, listofcomponents):
        # Parameters for vapor pressure (Extended Antoine)
        listofvalues = ['VAL1','VAL2','VAL3','VAL4','VAL5','VAL6','VAL7']  #Default values given in Aspen
        listA, listB, listC, listD, listE, listF, listG = [],[],[],[],[],[],[]
        for i in listofvalues:
            for j in listofcomponents:
                path_constants = r"\Data\Properties\Parameters\Pure Components\PLXANT-1\Input\{value}\PLXANT\{comp}".format(value= i, comp = j)
                if i == 'VAL1':
                    listA.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL2':
                    listB.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL3':
                    listC.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL4':
                    listD.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL5':
                    listE.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL6':
                    listF.append(float(Application.Tree.FindNode(path_constants).Value))
                elif i == 'VAL7':
                    listG.append(float(Application.Tree.FindNode(path_constants).Value))  
        return listA, listB, listC, listD, listE, listF, listG

    def variables_FEDI(self,listofcomponents): 
        # listofcomponents: list of components that are included in the simulation
        x_in1 , x_in2, x_ins, Ts = [] , [], [], [] 
        Dir_S1 = r"\Data\Streams\S1\Output\MOLEFRAC\MIXED\{comp}"
        Dir_S2  = r"\Data\Streams\S2\Output\MOLEFRAC\MIXED\{comp}"
        for i in listofcomponents:
            # Extract composition of the stream S1
            if Application.Tree.FindNode(Dir_S1.format(comp = i)).Value == None:
                x_in1.append(0.00)
            else:
                x_in1.append(Application.Tree.FindNode(Dir_S1.format(comp = i)).Value)
            # Extract composition of the stream S2
            if Application.Tree.FindNode(Dir_S2.format(comp = i)).Value == None:
                x_in2.append(0.00)
            else:
                x_in2.append(Application.Tree.FindNode(Dir_S2.format(comp = i)).Value)  
        F1 = Application.Tree.FindNode(r"\Data\Streams\S1\Output\MOLEFLMX\MIXED").Value
        F2 = Application.Tree.FindNode(r"\Data\Streams\S2\Output\MOLEFLMX\MIXED").Value
        T_in_1 = Application.Tree.FindNode(r"\Data\Streams\S1\Output\TEMP_OUT\MIXED").Value + 273.15 #[K]
        T_in_2 = Application.Tree.FindNode(r"\Data\Streams\S2\Output\TEMP_OUT\MIXED").Value + 273.15 #[K]
        #Mean Temperature between solvent and azeotropic mixture
        Ts = ((T_in_1+T_in_2)/2) 
        # Temperature of top of RDWC column
        x_ins= ((np.array(F1)*np.array(x_in1)   +   np.array(F2)*np.array(x_in2)) / (F1+F2)) 
        return x_ins,  Ts

    def Fedi_Column(self, block_tag, listofcomponents, NF, NR, x_in, T, d, nt, FlashPoint_val, IgnitionTemperature_val): 
        # Enthalpys of combustion from aspen in cal/mol
        listHcom = self.EnthalpyCombustion(listofcomponents)
        # Parameters for vapor pressure (Extended Antoine)
        listA, listB, listC, listD, listE, listF, listG = self.Antoine_values(listofcomponents)
        # Calculations of fedi for the column
        FlashPoint = sum(FlashPoint_val*x_in)
        AutoIgnitionTemp = sum(IgnitionTemperature_val*x_in)
        Vol = (np.pi * ( d/2)**2)*(1.2*0.61*(nt-2))  # [m3]
        Pcolumn = float(Application.Tree.FindNode(r"\Data\Blocks\{block}\Input\PRES1".format(block=block_tag)).Value*100)  # [bar a kPa]
        Mass =  float(Application.Tree.FindNode(r"\Data\Blocks\{block}\Output\BAL_MASI_TFL".format(block=block_tag)).Value/3600)  # [Kg/sec]
        Treb = Application.Tree.FindNode(r"\Data\Blocks\{block}\Output\BOTTOM_TEMP".format(block = block_tag)).Value + 273.15 # Celsius to Kelvyn
        #---------------F1,F2,F3,F4-----------------#
        Entalpy_comb = sum(abs(listHcom)*x_in )  # [cal/mol a J/mol]
        F1 = 0.1*(Mass*(Entalpy_comb))/3.148
        F2 = (6/3.148)*Pcolumn*Vol
        Psat_in = self.Antoine_function(listA,listB, listC, listD, listE, listF, listG, T)*100 # [Bar a kPa]
        VapPress = sum(Psat_in*x_in )
        F3 = (1e-3)*(1/Treb)*((Pcolumn-VapPress)**2)*Vol   # La temperatura es de la operación
        # Penalty 1
        if T > FlashPoint and T < 0.75*AutoIgnitionTemp: 
            pn1 = (1.45 +  1.75)/2
        elif T > 0.75*AutoIgnitionTemp:
            pn1 = 1.95
        else:
            pn1 = 1.1
        # Penalty 2
        if VapPress > 101.325 and Pcolumn > VapPress :
            pn2 = 1 + (Pcolumn-VapPress)*0.6/Pcolumn
            F = F2 + F3
        else:
            pn2 = 1 + (Pcolumn-VapPress)*0.4/Pcolumn
            F = F2
        if VapPress < 101.325 and 101.325 < Pcolumn:
            pn2 = 1 + (Pcolumn-VapPress)*0.2/Pcolumn
            F = F3
        else:
            pn2 = 1.1
            F = F3     
        # Penalty 4
        pn4 = 1 + 0.25*(np.array(NF).max() + np.array(NR).max())
        pn3,pn5,pn6 = 1,1,1
        Damage_Potential = (F1*pn1 + F*pn2)*(pn3*pn4*pn5*pn6) 
        return 4.76*(Damage_Potential**(1/3)) 

    def CO2(self,tags, NHV = 39771 , C = 86.5, alfa = 3.67):
        CO2 = []
        for i in range(len(tags)):
            block_tag = tags[i]
            Temp  = Application.Tree.FindNode(r"\Data\Blocks\{block}\Output\BOTTOM_TEMP".format(block = block_tag)).Value + 273.15 # Celsius to Kelvyn
            ReboilerDuty = Application.Tree.FindNode(r"\Data\Blocks\{block}\Output\REB_DUTY".format(block = block_tag)).Value*0.0041868    #cal/sec to kW
            ReboilerService = [433,457,527]  # Tempearture [K]
            LatenHeatService = [2085.03,1998.55,1697.79]    #KJ/Kg
            EnthalpyService = [2758.65,2780.06,2802.23]     #KJ/Kg
            if Temp < ReboilerService[0]:       #Use low pressure steam 
                L = LatenHeatService[0]
                Enthalpy = EnthalpyService[0]
            elif Temp < ReboilerService[1]:     #Use medium pressure steam
                L = LatenHeatService[1]
                Enthalpy = EnthalpyService[1]
            elif Temp < ReboilerService[2]:     #Use high pressure steam
                L = LatenHeatService[2]
                Enthalpy = EnthalpyService[2]
            #The boiler feed water is assumed to be at 100 °C with an enthalpy of 419 kJ/kg
            Tftb = 1800 + 273.15    #[K]
            Tstack = 160 + 273.15   #[K]
            To = 25+273.15          #[K]
            efficiency = (Tftb - To)/(Tftb - Tstack)
            Q_Fuel = (ReboilerDuty/L)*(Enthalpy - 419)*efficiency
            CO2_val = (Q_Fuel/NHV)*(C/100)*alfa
            CO2.append(CO2_val) # [kg/hr]
        return sum(np.array(CO2))*8760   # [kg/yr]

    def EnthalpyCombustion(self, listofcomponents):
        # Enthalpys of combustion from aspen in cal/mol
        listHcom = []
        for i in listofcomponents:
            #listHcom.append(float(Application.Tree.FindNode(r"\Data\Properties\Parameters\Pure Components\REVIEW-1\Input\VALUE\HCOM\{comp}".format(comp = i)).Value)/1000) #kcal/mol 
            dic = r"\Data\Properties\Parameters\Pure Components\REVIEW-1\Input\VALUE\HCOM\{comp}"
            Hcom = float(Application.Tree.FindNode(dic.format(comp = i)).Value)/1000  #cal/mol a kcal/mol 
            Hcom = Hcom*4184  #J/mol
            listHcom.append(Hcom)
        return  np.array(listHcom)

    def PSI(self, tags, listofcomponents, listLFL , listUFL, tags_inputs_1):
        # Enthalpys of combustion from aspen in cal/mol
        listHcom = self.EnthalpyCombustion(listofcomponents)
        # Calculate PSI for each stream
        Flows_values, LFL_T, UFL_T = [] , [] , []
        UFL_mix, LFL_mix = np.zeros(len(tags)), np.zeros(len(tags))
        Heating_values,Density_values,Pressure_values, Combustibility   = [] , [], [], []
        for i, c in enumerate(tags):
            T_stream = float(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\RES_TEMP".format(stream=c)).Value)    # Celsius
            Flows_values.append(float(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\TOT_FLOW".format(stream=c)).Value)) 
            # EXTRACT MOLAR COMPOSITION OF STREAM
            List_x = []
            for j, k in enumerate(listofcomponents):
                List_x.append(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\MOLEFRAC\MIXED\{comp}".format(stream=c,comp=k)).Value)
                LFL_T.append(listLFL[j]*(1 - 0.75*(T_stream - 25)/listHcom[j] ))
                UFL_T.append(listUFL[j]*(1 + 0.75*(T_stream - 25)/listHcom[j] ))
            # ENTHALPY OF COMBUSTION OF MIXTURE
            LFL_mix[i] = ( 1/ (sum(List_x[j]/LFL_T[i] for j in range(len(List_x))) ) )
            UFL_mix[i] = ( 1/ (sum(List_x[j]/UFL_T[i] for j in range(len(List_x))) ) )
            Combustibility.append( UFL_mix[i] - LFL_mix[i] )  # Calculate combustion for each stream}
            # Get average mass heating value
            Heating_values.append(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\RHOMX_MASS\MIXED".format(stream=c)).Value)
            Density_values.append(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\HMX_MASS\MIXED".format(stream=c)).Value)
            Pressure_values.append(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\PRES_OUT\MIXED".format(stream=c)).Value)
        # Convert to numpy
        Flows_values = np.array(Flows_values)
        Heating_values = np.array(Heating_values)
        Density_values = np.array(Density_values)
        Pressure_values = np.array(Pressure_values)
        Combustibility = np.array(Combustibility)
        # Extract molar flows of inputs streams equipment onethe entire flowsheet
        Flows_values_1 = []
        for i in tags_inputs_1:
            Flows_values_1.append(float(Application.Tree.FindNode(r"\Data\Streams\{stream}\Output\TOT_FLOW".format(stream=i)).Value)) 
        Flows_values_1 = np.array(Flows_values_1).sum()
        # CALCULATE WEIGTED FEDI
        PSI_nom, PSI_mass, PSI_total_mass = 0, 0 ,0
        for i, _ in enumerate(tags):
            A1 = (Heating_values[i]/np.average(Heating_values))
            A2 = (Density_values[i]/np.average(Density_values))
            A3 = (Pressure_values[i]/np.average(Pressure_values))
            A4 = (Combustibility[i]/np.average(Combustibility))
            PSI_total_mass +=  (Flows_values[i]/Flows_values_1)*100*A1*A2*A3*A4
            PSI_mass +=  (Flows_values[i]/Flows_values.mean())*100*A1*A2*A3*A4  #Adaptación de flujo
            PSI_nom += 100*A1*A2*A3*A4  #Adaptación de flujo
        return [PSI_nom, PSI_mass, PSI_total_mass]

    def constraints_eval(self, x):
        Constraint_List = []
        # Restricciones de la primera columna
        Constraint_List.append(x[0] < x[1])   # Feed Stage S1 debe ser menor a Feed Stage S2 
        Constraint_List.append(x[0] < x[2])   # Feed Stage S1 debe ser menor que B1 
        Constraint_List.append(x[1] < x[2])   # Feed Stage S2 debe ser menor que B1 
        # Restricciones de la segunda columna
        Constraint_List.append(x[2] < x[3])   # Las etapas de B1 deben ser menor a las de B2
        Constraint_List.append( (x[2] + x[5] +1) < x[3] )  
        # La suma de la etapa de inicio de la pared divisiorio 
        # más etapas del prefaacionador debe ser menor que las columnna principal, osea debe de caber el prefaccionador.
        Constraint_List.append(x[5] < x[3])  # la posicion de la pared divisoria debe ser menor que las etapas de B2
        Constraint_List.append(x[6] > x[5])  # la corriente lateral debe salir en la sección de la pared divisoria
        Constraint_List.append(x[6] < (x[2] + x[5] +1 ) )  # limite superior para restricción de arriba.
        return Constraint_List

    def Clean_Aspen(self):
        #-----------------------------Column B1-----------------------------#
        x = [2, 68, 70, 90, 5, 2, 55]
        # Update feed position of L2 and V3 and B1 Stages 
        B1_NSTAGE = r"\Data\Blocks\B1\Input\NSTAGE"
        Application.Tree.FindNode(B1_NSTAGE).Value = x[2]
        # Update feed position of stream S1 y S2
        FEED_STAGE_S1 = r"\Data\Blocks\B1\Input\FEED_STAGE\S1"
        FEED_STAGE_S2 = r"\Data\Blocks\B1\Input\FEED_STAGE\S2"
        Application.Tree.FindNode(FEED_STAGE_S1).Value = x[0]
        Application.Tree.FindNode(FEED_STAGE_S1).Value = x[1]

        FEED_STAGE_L2 = r"\Data\Blocks\B1\Input\FEED_STAGE\L2"
        FEED_STAGE_V3 = r"\Data\Blocks\B1\Input\FEED_STAGE\V3"
        Application.Tree.FindNode(FEED_STAGE_L2).Value = 1     # siempre va a entrar en la primera etapa
        Application.Tree.FindNode(FEED_STAGE_V3).Value = x[2]  # va a salir en la ultima etapa 

        PROD_STAGE_V2 =   r"\Data\Blocks\B1\Input\PROD_STAGE\V2"
        PROD_STAGE_L3 =   r"\Data\Blocks\B1\Input\PROD_STAGE\L3"
        Application.Tree.FindNode(r"\Data\Blocks\B1\Input\PROD_STAGE\V2").Value = 1    # siempre va a entrar en la primera etapa
        Application.Tree.FindNode(r"\Data\Blocks\B1\Input\PROD_STAGE\L3").Value = x[2]
        B1_INT_1 = r"\Data\Blocks\B1\Subobjects\Column Internals\INT-1\Input\CA_STAGE2\INT-1\INT"
        Application.Tree.FindNode(B1_INT_1).Value = x[2]
        B1_REACT_STAGE = r"\Data\Blocks\B1\Subobjects\Column Internals\INT-1\Input\REAC_STAGE2\#0"
        Application.Tree.FindNode(B1_REACT_STAGE).Value = x[2]       
         #-----------------------------Column B2-----------------------------#
        # Etapas totales
        B2_NSTAGE = Application.Tree.FindNode(r"\Data\Blocks\B2\Input\NSTAGE").Value
        Application.Tree.FindNode(r"\Data\Blocks\B2\Input\NSTAGE").Value = x[3]
        # Reflujo
        B2_BASIS_RR  = r"\Data\Blocks\B2\Input\BASIS_RR"
        Application.Tree.FindNode(B2_BASIS_RR).Value = x[4]

        # Alimentación de flujos
        FEED_STAGE_V2 = r"\Data\Blocks\B2\Input\FEED_STAGE\V2"
        Application.Tree.FindNode(FEED_STAGE_V2).Value = x[5]  # Posicion en la que inicia la pared divisoria
        FEED_STAGE_L3 = r"\Data\Blocks\B2\Input\FEED_STAGE\L3"
        Application.Tree.FindNode(FEED_STAGE_L3).Value = x[5] + x[2] + 1  # Posición + etapas de B1 + 1

        # Conexión de los flujos 
        PROD_STAGE_L2 = r"\Data\Blocks\B2\Input\PROD_STAGE\L2"
        Application.Tree.FindNode(PROD_STAGE_L2).Value  = x[5]

        PROD_STAGE_V3 = r"\Data\Blocks\B2\Input\PROD_STAGE\V3"
        Application.Tree.FindNode(PROD_STAGE_V3).Value  = x[5] + x[2] + 1

        #

        PROD_STAGE_5 = r"\Data\Blocks\B2\Input\PROD_STAGE\5"   #Posición de destilado
        Application.Tree.FindNode(PROD_STAGE_5).Value = 1
        
        PROD_STAGE_7 = r"\Data\Blocks\B2\Input\PROD_STAGE\7"
        Application.Tree.FindNode(PROD_STAGE_7).Value = x[6]   # Posición de donde sale la corriente lateral
        
        PROD_STAGE_8 = r"\Data\Blocks\B2\Input\PROD_STAGE\8"
        Application.Tree.FindNode(PROD_STAGE_8).Value = x[3]  # Posición de fondos (etapa final)
        
        B2_INT_1 = r"\Data\Blocks\B2\Subobjects\Column Internals\INT-1\Input\CA_STAGE2\INT-1\INT"
        Application.Tree.FindNode(B2_INT_1).Value = x[3] - 1 

    def Update_Aspen(self,x):
        # UPDATE PRESSURE OF THE ENTIRE PROCESS
        Application.Tree.FindNode(r"\Data\Blocks\B1\Input\PRES1").Value = round(x[9],2)
        Application.Tree.FindNode(r"\Data\Blocks\B2\Input\PRES1").Value = round(x[9],2)
        Application.Tree.FindNode(r"\Data\Blocks\C103\Input\PRES1").Value = round(x[9],2)
        # PRESSURE OF INPUT STREAMS
        Application.Tree.FindNode(r"\Data\Streams\1\Input\PRES\MIXED").Value = round(x[9],2)
        Application.Tree.FindNode(r"\Data\Streams\2\Input\PRES\MIXED").Value = round(x[9],2)
        #-----------------------------Column B1-----------------------------#

        # Update feed position of L2 and V3 and B1 Stages 
        B1_NSTAGE = r"\Data\Blocks\B1\Input\NSTAGE"
        Application.Tree.FindNode(B1_NSTAGE).Value = x[2]
        
        # Update feed position of stream S1 y S2
        FEED_STAGE_S1 = r"\Data\Blocks\B1\Input\FEED_STAGE\S1"
        FEED_STAGE_S2 = r"\Data\Blocks\B1\Input\FEED_STAGE\S2"
        Application.Tree.FindNode(FEED_STAGE_S1).Value = x[0]
        Application.Tree.FindNode(FEED_STAGE_S2).Value = x[1]
    
        
        FEED_STAGE_L2 = r"\Data\Blocks\B1\Input\FEED_STAGE\L2"
        FEED_STAGE_V3 = r"\Data\Blocks\B1\Input\FEED_STAGE\V3"
        Application.Tree.FindNode(FEED_STAGE_L2).Value = 1     # siempre va a entrar en la primera etapa
        Application.Tree.FindNode(FEED_STAGE_V3).Value = x[2]  # va a salir en la ultima etapa 

        PROD_STAGE_V2 =   r"\Data\Blocks\B1\Input\PROD_STAGE\V2"
        PROD_STAGE_L3 =   r"\Data\Blocks\B1\Input\PROD_STAGE\L3"
        Application.Tree.FindNode(r"\Data\Blocks\B1\Input\PROD_STAGE\V2").Value = 1    # siempre va a entrar en la primera etapa
        Application.Tree.FindNode(r"\Data\Blocks\B1\Input\PROD_STAGE\L3").Value = x[2]

        B1_INT_1 = r"\Data\Blocks\B1\Subobjects\Column Internals\INT-1\Input\CA_STAGE2\INT-1\INT"
        Application.Tree.FindNode(B1_INT_1).Value = x[2]

        B1_REACT_STAGE = r"\Data\Blocks\B1\Subobjects\Column Internals\INT-1\Input\REAC_STAGE2\#0"
        Application.Tree.FindNode(B1_REACT_STAGE).Value = x[2]
        
        #-----------------------------Column B2-----------------------------#
        # Etapas totales
        B2_NSTAGE = r"\Data\Blocks\B2\Input\NSTAGE"
        Application.Tree.FindNode(B2_NSTAGE).Value = x[3]
        # Reflujo
        B2_BASIS_RR  = r"\Data\Blocks\B2\Input\BASIS_RR"
        Application.Tree.FindNode(B2_BASIS_RR).Value = x[4]

        # Alimentación de flujos
        FEED_STAGE_V2 = r"\Data\Blocks\B2\Input\FEED_STAGE\V2"
        Application.Tree.FindNode(FEED_STAGE_V2).Value = x[5]  # Posicion en la que inicia la pared divisoria
        FEED_STAGE_L3 = r"\Data\Blocks\B2\Input\FEED_STAGE\L3"
        Application.Tree.FindNode(FEED_STAGE_L3).Value = x[5] + x[2] + 1  # Posición + etapas de B1 + 1

        # Conexión de los flujos 
        PROD_STAGE_L2 = r"\Data\Blocks\B2\Input\PROD_STAGE\L2"
        Application.Tree.FindNode(PROD_STAGE_L2).Value  = x[5]

        PROD_STAGE_V3 = r"\Data\Blocks\B2\Input\PROD_STAGE\V3"
        Application.Tree.FindNode(PROD_STAGE_V3).Value  = x[5] + x[2] + 1

        #

        PROD_STAGE_5 = r"\Data\Blocks\B2\Input\PROD_STAGE\5"   #Posición de destilado
        Application.Tree.FindNode(PROD_STAGE_5).Value = 1
        
        PROD_STAGE_7 = r"\Data\Blocks\B2\Input\PROD_STAGE\7"
        Application.Tree.FindNode(PROD_STAGE_7).Value = x[6]   # Posición de donde sale la corriente lateral
        
        PROD_STAGE_8 = r"\Data\Blocks\B2\Input\PROD_STAGE\8"
        Application.Tree.FindNode(PROD_STAGE_8).Value = x[3]  # Posición de fondos (etapa final)


        PROD_FLOW_L2 = r"\Data\Blocks\B2\Input\PROD_FLOW\L2"
        Application.Tree.FindNode(PROD_FLOW_L2).Value = x[7]

        PROD_FLOW_V3 = r"\Data\Blocks\B2\Input\PROD_FLOW\V3"
        Application.Tree.FindNode(PROD_FLOW_V3).Value = x[8]
        
        B2_INT_1 = r"\Data\Blocks\B2\Subobjects\Column Internals\INT-1\Input\CA_STAGE2\INT-1\INT"
        Application.Tree.FindNode(B2_INT_1).Value = x[3] - 1

    def Error_Run(self):
        return 1000000, 0.00001, 1000000, 1000000, 1000000
    
    def B1_Check(self,listofcomponents, nt1 ):
        x_in_check = []
        b1_dir = r"\Data\Blocks\B1\Output\X\{nts}\{comp}"
        for i in listofcomponents:
            x_in_check.append(Application.Tree.FindNode(b1_dir.format(nts = str(int(nt1)), comp = str(i) )) == None)
        return x_in_check

    def B2_Check(self,listofcomponents, nt2 ):
        x_in_check = []
        b2_dir = r"\Data\Blocks\B2\Output\X\{nts}\{comp}"
        for i in listofcomponents:
            x_in_check.append(Application.Tree.FindNode(b2_dir.format(nts = str(int(nt2)), comp = str(i) )) == None)
        return x_in_check

    def _evaluate(self, x, out, *args, **kwargs):
        x = np.array([x[f"x{k:01}"] for k in range(1,11)])
        listofcomponents = ['IB','NBA','ETOH','ETBE']
        #----------Constraints function----------#
        cnst = self.constraints_eval(x)
        if all(cnst) == True : 
            self.Clean_Aspen()
            self.Update_Aspen(x)
            #-Run problem -#
            Application.Engine.Run2()
            Run_Status_Dir = r"\Data\Results Summary\Run-Status\Output\UOSSTAT2"
            #First Check if Status return a value
            if Application.Tree.FindNode(Run_Status_Dir) == None:  # This means there was a problem
                TAC,XD1, Fedi, CO2_process, PSI_weigted = self.Error_Run()
            else: # Get the current status of the simulation
                Run_Status = Application.Tree.FindNode(Run_Status_Dir).Value
                B1Check= self.B1_Check(listofcomponents,x[0])
                B2Check= self.B2_Check(listofcomponents,x[6])
                if Run_Status == 9  or all(B1Check) == True or all(B2Check) == True:
                    TAC, XD1, Fedi, CO2_process, PSI_weigted = self.Error_Run()
                    Application.Reinit()  # Restart the simulation and clean the errors
                else:   # Simulation without errors  == 8 
                    B1_d = Application.Tree.FindNode(r"\Data\Blocks\B1\Subobjects\Column Internals\INT-1\Subobjects\Sections\INT\Input\CA_DIAM\INT-1\INT").Value
                    B2_d = Application.Tree.FindNode(r"\Data\Blocks\B2\Subobjects\Column Internals\INT-1\Subobjects\Sections\INT\Input\CA_DIAM\INT-1\INT").Value
                    ds = max(B1_d,B2_d)
                    #----------------#
                    #  Columna B2    #
                    #----------------#
                    nt = x[3] # Value of number of stages       
                    P_sys = x[9] #
                    CostColumn = self.Column(nt1= nt,ds= ds, P = P_sys )  # [USD/Yr]
                    #----------------#
                    # CONDENSADOR B2 #
                    #----------------#
                    COND_DUTY = r"\Data\Blocks\B2\Output\COND_DUTY"
                    TOP_TEMP = r"\Data\Blocks\B2\Output\TOP_TEMP"
                    CostCondenser = self.CondenserColumn(COND_DUTY,TOP_TEMP) # [USD/Yr]
                    #--------------#
                    # REBOILER  B2 #
                    #--------------#
                    REB_DUTY = r"\Data\Blocks\B2\Output\REB_DUTY"
                    BOTTOM_TEMP = r"\Data\Blocks\B2\Output\BOTTOM_TEMP"
                    CostReboiler = self.ReboilerColumn(REB_DUTY,BOTTOM_TEMP) # [USD/Yr]
                    TAC  = (CostColumn + CostCondenser +  CostReboiler)/1e6  

                    #------- Fedis  Blocks-----------#
                    #Inputs of the compounts 

                    # Isobutylene information https://pubchem.ncbi.nlm.nih.gov/compound/Isobutylene-13C4
                    # https://www.airgas.com/msds/001031.pdf

                    # Butene information 
                    # https://pubchem.ncbi.nlm.nih.gov/compound/Butane#section=LogP

                    # Ethanol information
                    # https://pubchem.ncbi.nlm.nih.gov/compound/Ethanol#section=Stability-Shelf-Life

                    # TERT-BUTYL-ETHYL-ETHER information
                    # https://pubchem.ncbi.nlm.nih.gov/compound/tert-Butyl-ethyl-ether

                    FlashPoint_val = np.array([-76.1,-60,14,-19]) +273.15             # [K] 
                    IgnitionTemperature_val = np.array([465,287,363,375 ]) + 273.15   # [K]
                    #NFPA values
                    NF = np.array([4,4,3,3])
                    NR = np.array([1,0,0,0])    
                    # Flammable limits
                    listLFL = np.array([1.8,1.9,3.3,1.23])   # % by volume
                    listUFL = np.array([9.6,8.5,19,7.7])   # % by volume

                    x_in, T = self.variables_FEDI(listofcomponents)

                    Fedi = self.Fedi_Column('B2', listofcomponents, NF, NR, x_in, T, ds, nt, FlashPoint_val, IgnitionTemperature_val)
                    #------------ CO2 --------------#
                    CO2_process = self.CO2(['B2'])
                    
                    tags = ['S1','S2','5','7','8']  # Streams tags
                    tags_1 = ['S1','S2']
                    PSI_weigted_vals = self.PSI(tags, listofcomponents, listLFL , listUFL, tags_1 )
                    PSI_weigted = PSI_weigted_vals[2] 
                    #------------ Constraints------------#
                    c1_mole_frac = r"\Data\Streams\8\Output\MOLEFRAC\MIXED\ETBE"
                    # Get the information of purities in the distillates from intensified column
                    XD1  = Application.Tree.FindNode(c1_mole_frac).Value
        else:
            # Unfeasible desing
            TAC, XD1, Fedi, CO2_process, PSI_weigted = self.Error_Run()
        out["F"] = [TAC, Fedi, CO2_process, PSI_weigted]                #Declare the functions
        out["G"] = [(-XD1 + 0.9990)]    #Declare the constraints (purity)


In [5]:
class objective_pressure():
    
    def __init__(self, funtion_val, number_pd):
        self.funtion_val = funtion_val
        self.number_pd = number_pd

    def save_df(self,res):
        # Extract best results 
        sol, fun, const = [], [], []
        for i, c in enumerate(res.history):
            sol.extend(c.opt.get("F"))
            fun.extend(c.opt.get("X"))
            const.extend(-c.opt.get("G")+0.9990)
        df1 = pd.DataFrame(data=sol , columns=['TAC', 'FEDI', 'CO2', 'PSI'])
        df2 = pd.DataFrame(data=fun )
        df3 = pd.DataFrame(data=const, columns=['xd1'])
        df =pd.concat([df1,df2,df3], axis= 1)
        filtered_values = np.where((df['TAC']<100) & (df['xd1']>0.9990) )
        df = df.loc[filtered_values]
        df = df.drop_duplicates()
        df.to_csv(r'results_intensified_with_pressure_run{number}.csv'.format(number=self.number_pd+1), index=False)  
        return df

    def __call__(self, trial):
        # Hyperparameters to be optimized
        gen = trial.suggest_int('#gen', 60, 95) 
        pop = trial.suggest_int('#pop', 250, 350) 
        ofs = trial.suggest_int('#ofs', 50, 70)
        # Load problem
        problem = IntensifiedSequence_Pressure()
        # Define the number of generations 
        termination = get_termination("n_gen", gen)
        algorithm = NSGA2(pop_size=pop, n_offsprings=ofs, sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination())
        res = minimize(problem,algorithm,termination,seed=42,save_history=True,verbose=False)
        df = self.save_df(res)
        return  df.iloc[:,self.funtion_val].min()  # return minimum value encountered


In [9]:
study_name = 'NSGAII_intensified_pressure'
storage_traj = f'sqlite:///{study_name}.db'

In [10]:
study = opt.create_study(direction='minimize', study_name=study_name, storage=storage_traj, load_if_exists=True)

[I 2024-09-08 15:46:57,980] A new study created in RDB with name: NSGAII_intensified_pressure


In [8]:
# Use thisa command to eliminate the study if the simulation have some errors and the trial fails 
opt.delete_study(study_name=study_name, storage=storage_traj)

In [11]:
ciclos = 15
val_function = 0 # TAC position
for i in tqdm(range(ciclos)):    
    file = 'ETBE_Process_November_16_2022_RDWC.bkp'
    aspen_Path = os.path.abspath(file)
    Application = win32.Dispatch('Apwn.Document')
    Application.InitFromArchive2(aspen_Path)
    Application.Visible  = False
    Application.SuppressDialogs = True
    Application.Engine.Run2()
    time.sleep(5)   # Time to allow aspen to get a result.
    study.optimize(objective_pressure(funtion_val=val_function, number_pd= i), n_trials= 1)
    Application.Quit(aspen_Path)
# Best hyperparameters    
print(study.best_params)

  0%|          | 0/15 [00:00<?, ?it/s]

[I 2024-09-08 18:46:29,665] Trial 0 finished with value: 1.3744890887899097 and parameters: {'#gen': 86, '#pop': 332, '#ofs': 57}. Best is trial 0 with value: 1.3744890887899097.
[I 2024-09-08 23:02:45,632] Trial 1 finished with value: 1.309152027948601 and parameters: {'#gen': 93, '#pop': 275, '#ofs': 58}. Best is trial 1 with value: 1.309152027948601.
[I 2024-09-08 23:28:05,442] Trial 2 finished with value: 1.5725212082361975 and parameters: {'#gen': 73, '#pop': 318, '#ofs': 69}. Best is trial 1 with value: 1.309152027948601.
[I 2024-09-09 04:47:50,367] Trial 3 finished with value: 1.2536300066537378 and parameters: {'#gen': 93, '#pop': 333, '#ofs': 69}. Best is trial 3 with value: 1.2536300066537378.
[I 2024-09-09 05:20:32,805] Trial 4 finished with value: 1.4686352125860327 and parameters: {'#gen': 74, '#pop': 297, '#ofs': 68}. Best is trial 3 with value: 1.2536300066537378.
[I 2024-09-09 06:36:05,526] Trial 5 finished with value: 1.5526873590564052 and parameters: {'#gen': 62, '#p

{'#gen': 95, '#pop': 349, '#ofs': 66}
